In [58]:
import numpy as np
from scipy.special import erf

La base de funciones a considerar es
$$ \chi_1 = \psi_+ \alpha \,, \quad
\chi_2 = \psi_+ \beta \,, \quad
\chi_3 = \psi_- \alpha \,, \quad
\chi_4 = \psi_- \beta \,,
$$

donde $\psi_\pm = c_\pm (\varphi_A \pm \varphi_B)$ con

$$ c_\pm = \frac{1}{\sqrt{2 (1 \pm S)}}\,, $$

la constante de normalización y

$$ \varphi = \varphi(r, \theta, \phi) = R_{1\mathrm{s}}^k(r) Y_{00} (\theta, \phi) \,$$

las funciones espaciales.

En esto último $R_{1\mathrm{s}}^{k}(\zeta, \vec{d}, \vec{\alpha}, r) = \sum_{i=1}^{k} d_i R_{0}^{\mathrm{GTO}} (\zeta^2 \alpha_i, r)$ donde

$$ R_0^{\mathrm{GTO}} (\alpha, r) = \frac{2(2\alpha)^{3/4}}{\pi^{1/4}} \exp(-\alpha r^2) $$

<div class="alert alert-block alert-info">
        <strong>Integral de traslape, $S$</strong>
</div>

Aquí $S = \braket{\varphi_A|\varphi_B} = \braket{\varphi_B|\varphi_A}$ es la integral de traslape dada por

$$ \begin{align}
    S 
    &= \int R_{1s}^k (r_A) Y_{00}(\theta, \phi) \, R_{1s}^k (r_B) Y_{00}(\theta, \phi) \,\mathrm{d}V \\
    &= \int R_{1s}^k (r_A) \, R_{1s}^k (r_B) \,\mathrm{d}\mathbf{r} \int Y_{00}(\theta, \phi)^2 d\Omega \\
    &= \int \sum_{i=1}^{k} \left[ d_i \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \exp(-\alpha_i r^2_A) \right]
    \sum_{j=1}^{k} \left[ d_j \frac{2(2\beta_j)^{3/4}}{\pi^{1/4}} \exp(- \beta_j \, r^2_B) \right] \,\mathrm{d}\mathbf{r} \\
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \beta_j)^{3/4} \, \int \exp(-\alpha_i r^2_A) \exp(-\beta_j \, r^2_B) \,\mathrm{d}\mathbf{r}
\end{align}
$$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ \int \exp(-a r^2_A) \exp(-b \, r^2_B) \,\mathrm{d}\mathbf{r}
= \left( \frac{\pi}{a + b} \right)^{3/2} \exp\left( -\frac{a b}{a + b} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
$$

No obstante, ellos integran sobre todo el espacio, así que hay un factor de $4\pi$ adicional. Así pues, en nuestro caso se obtendría

$$ \int \exp(-a r^2_A) \exp(-b \, r^2_B) \,\mathrm{d}\mathbf{r}
= \frac{1}{4\pi} \left( \frac{\pi}{a + b} \right)^{3/2} \exp\left( -\frac{a b}{a + b} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
$$

El término exponencial aparece a lo largo de las integrales por calcular, motivo por el cual considere las siguientes convenciones:

$$ \mathrm{arg} = -\frac{a b}{a + b} |\mathbf{R}_A - \mathbf{R}_B|^2 \,, \qquad
K = \exp(\mathrm{arg}) \,.$$

In [59]:
RA = np.array([0,0,0])
RB = np.array([2,0,0])
np.square(np.linalg.norm(RA-RB))

np.float64(4.0)

In [60]:
def arg(a, b, RA, RB):
    """
    z : exponente orbital de Slater
    """
    p = a+b
    mu = a*b/p
    RAB2 = np.square(np.linalg.norm(RA-RB))
    return -mu*RAB2

In [61]:
def K(a, b, RA, RB):
    return np.exp(arg(a, b, RA, RB))

La expresión final de la integral de traslape $S$ es

$$ \boxed{ S = 2^{3/2} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j
    \frac{(\alpha_i \, \beta_j)^{3/4}}{(\alpha_i + \beta_j)^{3/2}}
    \exp\left( -\frac{\alpha_i \beta_j}{\alpha_i + \beta_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) }
$$

In [62]:
def SAB(d, a, RA, RB):
    """
        Integral de traslape S
    z : exponente orbital de Slater
    """
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 3/4) / np.power(a[i]+a[j], 3/2) * K(a[i], a[j], RA, RB)
            # print(i,j)
    return np.power(2, 3/2) * suma

In [63]:
d = np.array([1,2,3])
a = np.array([1,2,3])
RA = np.array([0,0,0])
RB = np.array([1.4,0,0])
SAB(d, a, RA, RB)

np.float64(4.62538896564451)

<div class="alert alert-block alert-info">
<strong>Coeficiente de normalización, $c_{\pm}$</strong>
</div>

In [64]:
def cPM(d, a, RA, RB, signo):
    S = SAB(d, a, RA, RB)
    if signo == 'plus':
        return 1/np.sqrt(2*(1+S))
    else:
        return 1/np.sqrt(2*(1-S))

<div class="alert alert-block alert-info">
<strong>Término $f_1^{(AB)}$</strong>
</div>

Términos $f_{1}^{(AB)}$, de la forma:

$$ \begin{align*}
    f_1^{(AB)}
    &= \int \varphi_A \left( -\frac{\nabla^2}{2} \right) \varphi_B \, \mathrm{d}V \\
    &= \int R_{1s}^k (r_A) Y_{00}(\theta, \phi) \left( -\frac{\nabla^2}{2} \right) R_{1s}^k (r_B) Y_{00}(\theta, \phi) \, \mathrm{d}V \\
    &= \int \sum_{i=1}^{k} \left[ d_i \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \exp(-\alpha_i r^2_A) \right]
        \left( -\frac{\nabla^2}{2} \right)
        \sum_{j=1}^{k} \left[ d_j \frac{2(2\beta_j)^{3/4}}{\pi^{1/4}} \exp(-\beta_j r^2_B) \right] \mathrm{d}\mathbf{r}
        \, \int  Y_{00}^2 \, \mathrm{d}\Omega \\
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \beta_j)^{3/4}
        \int \exp(-\alpha_i \, r^2_A)
        \left( -\frac{\nabla^2}{2} \right)
        \exp(-\beta_j \, r^2_B) \, \mathrm{d}\mathbf{r}
\end{align*} $$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ (A|-\tfrac{\nabla^2}{2}|B)
    = \int \exp(-\alpha r^2_A) \left( -\frac{\nabla^2}{2} \right) \exp(-\beta r^2_A) \mathrm{d}\mathbf{r}
    = \frac{\alpha \beta}{\alpha + \beta} \left[ 3 - \frac{2 \alpha \beta}{\alpha + \beta} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha + \beta} \right]^{3/2} \, \exp \left[ -\frac{\alpha \beta}{(\alpha + \beta)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]$$

No obstante, ellos integran sobre todo el espacio, así que hay un factor de $4\pi$ adicional. En cuyo caso, nosotros obtendríamos

$$ (A|-\tfrac{\nabla^2}{2}|B) = \int \exp(-\alpha r^2_A) \left( -\frac{\nabla^2}{2} \right) \exp(-\beta r^2_A) \mathrm{d}\mathbf{r}
= \frac{1}{4\pi} \frac{\alpha \beta}{\alpha + \beta} \left[ 3 - \frac{2 \alpha \beta}{\alpha + \beta} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha + \beta} \right]^{3/2} \, \exp \left[ -\frac{\alpha \beta}{(\alpha + \beta)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]$$

La expresión final para el término $f_1^{(AB)}$ es

$$\begin{align*}
f_1^{(AB)}
    &= \frac{1}{4\pi} \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \beta_j)^{3/4}
    \frac{\alpha_i \beta_j}{\alpha_i + \beta_j} \left[ 3 - \frac{2 \alpha_i \beta_j}{\alpha_i + \beta_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \left[ \frac{\pi}{\alpha_i + \beta_j} \right]^{3/2} \, \exp \left[ -\frac{\alpha_i \beta_j}{(\alpha_i + \beta_j)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \\
    &= \boxed{ 2^{3/2} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j \frac{(\alpha_i \beta_j)^{7/4}}{(\alpha_i + \beta_j)^{5/2}}
    \left[ 3 - \frac{2 \alpha_i \beta_j}{\alpha_i + \beta_j} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] \, \exp \left[ -\frac{\alpha_i \beta_j}{(\alpha_i + \beta_j)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] }
\end{align*}
$$

<br>

In [65]:
def f1AB(d, a, RA, RB):
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 7/4) / np.power(a[i]+a[j], 5/2) * (3 + 2*arg(a[i], a[j], RA, RB)) * K(a[i], a[j], RA, RB)
            # el signo + en 2*arg(...) se debe a que arg(...) ya considera el negativo
    return np.power(2, 3/2) * suma

<div class="alert alert-block alert-info">
<strong>Término $f_1^{(AA)}$</strong>
</div>

De este último es posible obtener el término $f_{1}^{(AA)}$ el cuál no sería otra cosa más que

$$ \boxed{f_1^{(AA)} = 3 (2^{3/2}) \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j \frac{(\alpha_i \beta_j)^{7/4}}{(\alpha_i + \beta_j)^{5/2}}} $$

In [66]:
def f1AA(d, a):
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma += d[i]*d[j] * np.power(a[i]*a[j], 7/4) / np.power(a[i]+a[j], 5/2)
            # el signo + en 2*arg(...) se debe a que arg(...) ya considera el negativo
    return 3*np.power(2, 3/2)*suma

<div class="alert alert-block alert-info">
<strong>Término $f_2^{(AB)}$</strong>
</div>

Términos $f_{2}^{(AB)}$, de la forma:

$$ \begin{align*}
    f_2^{(AB)}
    &= \int \varphi_A \left( \sum_C -\frac{Z_C}{r_C} \right) \varphi_B \, \mathrm{d}V \\
    &= \int R_{1s}^k (r_A) Y_{00}(\theta, \phi) \left( \sum_C -\frac{Z_C}{r_C} \right) R_{1s}^k (r_B) Y_{00}(\theta, \phi) \, \mathrm{d}V \\
    &= \int \sum_{i=1}^{k} \left[ d_i \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \exp(-\alpha_i r^2_A) \right]
        \left( \sum_C -\frac{Z_C}{r_C} \right)
        \sum_{j=1}^{k} \left[ d_j \frac{2(2\beta_j)^{3/4}}{\pi^{1/4}} \exp(-\beta_j r^2_B) \right] \mathrm{d}\mathbf{r}
        \, \int  Y_{00}^2 \, \mathrm{d}\Omega \\
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \beta_j)^{3/4}
        \sum_C \int \exp(-\alpha_i \, r^2_A)
        \left( -\frac{Z_C}{r_C} \right)
        \exp(-\beta_j \, r^2_B) \, \mathrm{d}\mathbf{r}
\end{align*} $$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ (A|-\frac{Z_C}{r_C}|B)
    = \int \exp(-\alpha \, r^2_A) \left( -\frac{Z_C}{r_C} \right) \exp(-\beta \, r^2_B) \mathrm{d}\mathbf{r}
    = -\frac{2\pi}{\alpha + \beta} Z_C \, \exp \left[ -\frac{\alpha \beta}{(\alpha + \beta)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    F_0 \Big[ (\alpha+\beta) |\mathbf{R}_P - \mathbf{R}_C|^2 \Big]
$$

donde $F_0(t)$ es

$$ F_0(t) = \frac{1}{2} \left( \frac{\pi}{t} \right)^{1/2} \mathrm{erf}(t^{1/2}) \,;
\quad
\text{donde} \quad \mathrm{erf}(z) = \frac{2}{\sqrt{\pi}} \int_0^{z} e^{-t^2} \, \mathrm{d}t
$$

y $\mathbf{R}_P$ es

$$ \mathbf{R}_P = \frac{\alpha \mathbf{R}_A + \beta \, \mathbf{R}_B}{\alpha + \beta} \,.$$

No obstante, ellos integran sobre todo el espacio, así que hay un factor de $4\pi$ adicional. En cuyo caso, nosotros obtendríamos

$$ (A|-\frac{Z_C}{r_C}|B)
    = \int \exp(-\alpha \, r^2_A) \left( -\frac{Z_C}{r_C} \right) \exp(-\beta \, r^2_B) \mathrm{d}\mathbf{r}
    = -\frac{Z_C}{2(\alpha + \beta)} \, \exp \left[ -\frac{\alpha \beta}{(\alpha + \beta)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    F_0 \Big[ (\alpha+\beta) |\mathbf{R}_P - \mathbf{R}_C|^2 \Big]
$$

In [67]:
def F0(t):
    return (1/2) * np.sqrt(np.pi/t) * erf(np.sqrt(t))

In [68]:
def RP(a:int, b:int, RA, RB):
    """
    a : valor numérico
    b : valor numérico
    """
    return (a*RA+b*RB)/(a+b)

La expresión final para el elemento $f_2^{(AB)}$ es

$$ \begin{align*}
f_2^{(AB)}
    &= \frac{2^{7/2}}{\pi^{1/2}}
        \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \beta_j)^{3/4}
        \sum_C -\frac{Z_C}{2(\alpha_i + \beta_j)} \, \exp \left[ -\frac{\alpha_i \beta_j}{(\alpha_i + \beta_j)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
        F_0 \Big[ (\alpha_i + \beta_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \Big] \\
    &= \boxed{\frac{2^{3/2}}{\pi^{1/2}}
        \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j
        \frac{(\alpha_i \, \beta_j)^{3/4}}{\alpha_i + \beta_j}
        \exp \left[ -\frac{\alpha_i \beta_j}{(\alpha_i + \beta_j)} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
        \sum_C -Z_C \, F_0 \Big[ (\alpha_i + \beta_j) |\mathbf{R}_P - \mathbf{R}_C|^2 \Big]}
\end{align*}
$$

<br>

In [69]:
def f2AB(d, a, RA, RB, Z):
    """
        Suma sobre núcleos
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    a : vector de coeficientes Gaussianos [a1, a2, a3]
    """
    R = [RA, RB]
    RAB2 = np.square(np.linalg.norm(RA-RB))
    
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            Rp = RP(a[i], a[j], RA, RB)
            suma_ZC = 0
            for C in range(len(Z)):
                RPC2 = np.square(np.linalg.norm(Rp-R[C]))
                suma_ZC += -Z[C] * F0((a[i]+a[j])*RPC2)
                # print(i,j,C)
            suma += d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j]) * K(a[i], a[j], RA, RB) * suma_ZC
    return np.power(2, 3/2) / np.sqrt(np.pi) * suma

In [70]:
d = np.array([0,0,0])
a = np.array([0,0,0])
RA = np.array([0,0,0])
RB = np.array([0,0,0])
Z = [1,2]
f2AB(d, a, RA, RB, Z)

C:\Users\rodri\AppData\Local\Temp\ipykernel_3176\2407126611.py:6: RuntimeWarning: invalid value encountered in divide
  return (a*RA+b*RB)/(a+b)
C:\Users\rodri\AppData\Local\Temp\ipykernel_3176\3734718454.py:20: RuntimeWarning: invalid value encountered in scalar divide
  suma += d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j]) * K(a[i], a[j], RA, RB) * suma_ZC
C:\Users\rodri\AppData\Local\Temp\ipykernel_3176\3664876648.py:6: RuntimeWarning: invalid value encountered in scalar divide
  mu = a*b/p


np.float64(nan)

<div class="alert alert-block alert-info">
<strong>Término $f_2^{(AA)}$</strong>
</div>

Términos $f_{2}^{(AA)}$, de la forma:

$$ \begin{align*}
    f_2^{(AA)}
    &= \int \varphi_A \left( \sum_C -\frac{Z_C}{r_C} \right) \varphi_A \, \mathrm{d}V \\
    &= \int R_{1s}^k (r_A) Y_{00}(\theta, \phi) \left( \sum_C -\frac{Z_C}{r_C} \right) R_{1s}^k (r_A) Y_{00}(\theta, \phi) \, \mathrm{d}V \\
    &= \int \sum_{i=1}^{k} \left[ d_i \frac{2(2\alpha_i)^{3/4}}{\pi^{1/4}} \exp(-\alpha_i r^2_A) \right]
        \left( \sum_C -\frac{Z_C}{r_C} \right)
        \sum_{j=1}^{k} \left[ d_j \frac{2(2\alpha_j)^{3/4}}{\pi^{1/4}} \exp(-\alpha_j r^2_A) \right] \mathrm{d}\mathbf{r}
        \, \int  Y_{00}^2 \, \mathrm{d}\Omega \\
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \alpha_j)^{3/4}
        \int \exp(-\alpha_i \, r^2_A)
        \left( \sum_C -\frac{Z_C}{r_C} \right)
        \exp(-\alpha_j \, r^2_A) \, \mathrm{d}\mathbf{r} \\
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \alpha_j)^{3/4}
        \left[ \int \exp(-\alpha_i \, r^2_A)
        \left( -\frac{Z_A}{r_A} + \sum_{C \neq A} -\frac{Z_C}{r_C} \right)
        \exp(-\alpha_j \, r^2_A) \, \mathrm{d}\mathbf{r} \right] \\
\end{align*} $$

Para la primer integral se sigue que

$$ (A |  -\frac{Z_A}{r_A} | A)
    = \int \exp(-\alpha_i \, r^2_A) \left( -\frac{Z_A}{r_A} \right) \exp(-\alpha_j \, r^2_A) \, \mathrm{d}\mathbf{r}
    = -Z_A \int \frac{1}{r_A} \exp \Big[ -(\alpha_i + \alpha_j) \, r^2_A \Big] \, \mathrm{d}\mathbf{r}
    = -\frac{Z_A}{2(\alpha_i + \alpha_j)}
$$

Para la segunda integral se sigue que

$$ (A |  -\frac{Z_C}{r_C} | A)
    = \int \exp(-\alpha_i \, r^2_A) \left( \sum_{C \neq A} -\frac{Z_C}{r_C} \right) \, \exp(-\alpha_j \, r^2_A) \, \mathrm{d}\mathbf{r}
    = \sum_{C\neq A} \int \exp(-\alpha_i \, r^2_A) \left( -\frac{Z_C}{r_C} \right) \exp(-\alpha_j \, r^2_A) \, \mathrm{d}\mathbf{r}
$$

lo cual no es más un caso de la integral $(A | -\frac{Z_C}{r_C} | B)$.

Es decir,

$$ (A |  -\frac{Z_C}{r_C} | A)
    = \sum_{C\neq A} -\frac{Z_C}{2(\alpha_i + \alpha_j)} \, F_0 \Big[ (\alpha_i + \alpha_j) |\mathbf{R}_A - \mathbf{R}_C|^2 \Big]
$$

La expresión final para el término $f_2^{(AA)}$ será

$$ \begin{align*}
f_2^{(AA)}
    &= \frac{2^{7/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j (\alpha_i \, \alpha_j)^{3/4}
    \left[ -\frac{Z_A}{2(\alpha_i + \alpha_j)}
    + \sum_{C\neq A} -\frac{Z_C}{2(\alpha_i + \alpha_j)} \, F_0 \Big[ (\alpha_i + \alpha_j) |\mathbf{R}_A - \mathbf{R}_C|^2 \Big]
    \right] \\
    &= \frac{2^{3/2}}{\pi^{1/2}} \sum_{i=1}^{k} \sum_{j=1}^{k} d_i d_j \frac{(\alpha_i \, \alpha_j)^{3/4}}{\alpha_i + \alpha_j}
    \left[ -Z_A + \sum_{C\neq A} - Z_C \, F_0 \Big[ (\alpha_i + \alpha_j) |\mathbf{R}_A - \mathbf{R}_C|^2 \Big]
    \right]
\end{align*}
$$

<br>

In [71]:
def f2AA(d, a, RA, RB, Z):
    """
        Suma sobre núcleos
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    """
    RAB2 = np.square(np.linalg.norm(RA-RB))
    ZA = Z[0]
    ZB = Z[1]
    
    suma = 0
    k = len(d)
    for i in range(k):
        for j in range(k):
            suma_ZC = -ZB * F0((a[i]+a[j])*RAB2)
            suma += d[i]*d[j] * np.power(a[i]*a[j], 3/4) / (a[i]+a[j]) * (-ZA + suma_ZC)
    return np.power(2, 3/2) / np.sqrt(np.pi) * suma

<div class="alert alert-block alert-info">
<strong>Elemento de matriz $f_{pp}$</strong>
</div>

$$ f_{pp} = 2 |c_\pm|^2 \left[ f_{1}^{(AA)} \pm f_{1}^{(AB)} + f_{2}^{(AA)} \pm f_{2}^{(AB)} \right] $$

In [91]:
def fpp(p, d, a, RA, RB, Z):
    """
    Z : vector de cargas nucleares [ZA, ZB, ..., ZC] donde type(ZA) = float
    """
    t1AA = f1AA(d,a)
    t1AB = f1AB(d, a, RA, RB)
    t2AA = f2AA(d, a, RA, RB, Z)
    t2AB = f2AB(d, a, RA, RB, Z)

    if p in [1,2]:
        c2 = np.square(cPM(d, a, RA, RB, 'plus')) 
        return 2*c2*(t1AA + t1AB + t2AA + t2AB)
    else:
        c2 = np.square(cPM(d, a, RA, RB, 'minus'))
        return 2*c2*(t1AA - t1AB + t2AA - t2AB)

In [92]:
# Parámetros Gaussianos obtenidos en `(1) STO-3G.ipynb` para zeta=1.24
dSeg = np.array([0.44471812476789035, 0.5352716544572346, 0.1543000507808527])
aSeg = np.array([0.16887939463273338, 0.6240343336327064, 3.4256944279866635])
aSeg = np.array([0.10983311305458726, 0.40584959263313375, 2.227949029647934])

dSte = np.array([0.444635, 0.535328, 0.154329])
aSte = np.array([0.109818, 0.405771, 2.22766])

RA = np.array([0, 0, 0])
RB = np.array([1.401, 0, 0]) # distancia interatómica de 1.4
ZA, ZB = 1, 1 # carga nuclear del hidrógeno
Z = [ZA, ZB]

In [93]:
f11Seg = fpp(1, dSeg, aSeg, RA, RB, Z)
f33Seg = fpp(3, dSeg, aSeg, RA, RB, Z)

print('f11 & f22:', f11Seg)
print('f33 & f44:', f33Seg)

f11 & f22: -0.38796370395015933
f33 & f44: 0.28231619430235233


In [94]:
f11Ste = fpp(1, dSte, aSte, RA, RB, Z)
f33Ste = fpp(3, dSte, aSte, RA, RB, Z)

print('f11 & f22:', f11Ste)
print('f33 & f44:', f33Ste)

f11 & f22: -0.3879647165768662
f33 & f44: 0.2823147754129697
